In [1]:
import pandas as pd
import numpy as np
import scipy


In [2]:
def to_list(words):
    if words == '[]':
        return []
    else:
        words = words.strip('"')
        words = words.strip('[')
        words = words.strip(']')
        words = words.split(',')
        words = [w.lstrip(' ') for w in words]
        words = [w.strip("'") for w in words]
        words = [w for w in words if w != '']

        return words


In [3]:
df_train = pd.read_csv('./dataset/2_with_cleaned_text/train.min.csv')
df_train['words'] = df_train['words'].apply(to_list)

X_train = df_train['words'].copy()
X_train = X_train.apply(lambda words: ' '.join(words))

df_test = pd.read_csv('./dataset/2_with_cleaned_text/test.min.csv')
df_test['words'] = df_test['words'].apply(to_list)

X_test = df_test['words'].copy()
X_test = X_test.apply(lambda words: ' '.join(words))


In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(lowercase=False, norm='l2', sublinear_tf=True)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

scipy.sparse.csr_matrix.sort_indices(X_train_tfidf)
scipy.sparse.csr_matrix.sort_indices(X_test_tfidf)


In [5]:
words_train = []
words_test = []

for i in range(X_train.shape[0]):
    words_train.append(X_train_tfidf[i].toarray().reshape((6633,)))
for i in range(X_test.shape[0]):
    words_test.append(X_test_tfidf[i].toarray().reshape((6633,)))


In [6]:
words_train = pd.Series(words_train)
words_test = pd.Series(words_test)


In [7]:
del df_train['words']
del df_test['words']


In [8]:
df_train = pd.concat([df_train, words_train.to_frame(name='words')], axis=1)
df_test = pd.concat([df_test, words_test.to_frame(name='words')], axis=1)


In [9]:
df_train['category'] = df_train['category']
df_test['category'] = df_test['category']


In [10]:
df_train.to_parquet('./dataset/3_with_tfidf/train.parquet', engine='pyarrow')
df_test.to_parquet('./dataset/3_with_tfidf/test.parquet', engine='pyarrow')

